Open the file `Porphyrin absorption.csv` in a text editor and determine its format. Import the data in the notebook using `numpy.loadtxt`. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
%matplotlib ipympl
    
data = np.loadtxt('Porphyrin absorption.csv', skiprows = 1,\
                  delimiter=',') 

wavelength = data[:,0]
absorbance = data[:,1]

2) Plot the spectrum, add axes labels and so on.

In [ ]:
fig,ax = plt.subplots(1,1)
ax.plot(wavelength,absorbance)
ax.set_xlabel('wavelength (nm)')
ax.set_ylabel('absorbance')
plt.show()

3) Start by fitting the spectrum with a single Gaussian function centered around the largest peak, and see the result.


In [ ]:
def model(x, A, mu, sig):
    return A/(np.sqrt(2*np.pi)*sig)*np.exp(-np.power((x - mu)/sig,2)/2)

x = np.linspace(400,700,1000)
p0 = [20,515,5]
iMin = 74
iMax = 134
popt, pcov = curve_fit(model, wavelength[iMin:iMax], absorbance[iMin:iMax], p0)

fitted_curve = model(x,*popt)
print(popt)

fig, ax = plt.subplots()
ax.plot(wavelength,absorbance,'.',label='Dati')
ax.plot(x, model(x,*p0),'g-',label='Modello')
ax.plot(x, fitted_curve, 'r-',label='Fit')
ax.set_xlabel('wavelengh (nm)')
ax.set_ylabel('absorbance')
ax.legend()
# ax[0].axvline(x=500)
plt.show()
